In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import GrubbsTest

In [ ]:
# Meter premise srvc_addr_2_nm field
srvc_addr_2_nms_EVs = [
    '306 E BEECHWOLD BLVD', # 2 Teslas, I believe
    '248 E BEAUMONT RD',    # Chevy Volt
    '4935 OLENTANGY BLVD',  # Tesla
    '60 W JEFFREY PL',      # Not sure, appears to be charge station
    '135 HOWARD AVE'        # Chevy Volt (Mike)
]

In [ ]:
srvc_addr_2_nms_neighbors = [
    '312 E BEECHWOLD BLVD', 
    '318 E BEECHWOLD BLVD', 
    
    '242 E BEAUMONT RD', 
    '256 E BEAUMONT RD', 
    
    '4925 OLENTANGY BLVD', 
    '4895 OLENTANGY BLVD', 
    
    '66 W JEFFREY PL', 
    '44 W JEFFREY PL', 
    
    '115 HOWARD AVE', 
    '121 HOWARD AVE'
]

In [ ]:
mp = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function_kwargs= dict(
        cols_of_interest = TableInfos.MeterPremise_TI.std_columns_of_interest + ['srvc_addr_2_nm', 'serv_city_ad'], 
        srvc_addr_2_nms = srvc_addr_2_nms_EVs+srvc_addr_2_nms_neighbors, 
        state='OH'
    ), 
)

In [ ]:
mp_df = mp.df.copy()

In [ ]:
mp_df

In [ ]:
date_0 = '2023-01-01'
date_1 = '2023-02-28'

In [ ]:
ami = AMINonVee(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function_kwargs= dict(
        cols_of_interest = TableInfos.AMINonVee_TI.std_columns_of_interest, 
        serial_numbers = mp_df['mfr_devc_ser_nbr'].tolist(), 
        date_range = [date_0, date_1], 
        state='OH'
    ), 
    init_df_in_constructor=True
)

In [ ]:
ami_df = ami.df.copy()
ami_df = ami_df[ami_df['aep_srvc_qlty_idntfr']=='TOTAL']

In [ ]:
ami_df

In [ ]:
ami_df = AMINonVee.perform_std_initiation_and_cleaning(ami_df)

In [ ]:
ami_df.dtypes

In [ ]:
ami_df_resamples = AMINonVee.build_time_resampled_dfs(ami_df, base_freq='15T', freqs=['H', '2H', '12H', 'D', 'W'])

In [ ]:
ami_df_resamples['2H'].keys()

In [ ]:
ami_df[['starttimeperiod', 'starttimeperiod_utc']]

In [ ]:
# fig, ax = Plot_General.default_subplots()
# sns.lineplot(ax=ax, x='starttimeperiod', y='value', hue='serialnumber', data=ami_df)

In [ ]:
tmp_df = ami_df[
    (ami_df['serialnumber']=='876122177') &
    (ami_df['starttimeperiod_utc']>pd.to_datetime('2023-02-01')) &
    (ami_df['starttimeperiod_utc']<pd.to_datetime('2023-02-07')) &
    (ami_df['aep_srvc_qlty_idntfr']=='TOTAL')
]

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='starttimeperiod_utc', y='value', data=tmp_df)

In [ ]:
tmp_df = ami_df[
    (ami_df['starttimeperiod_utc']>pd.to_datetime('2023-02-01')) &
    (ami_df['starttimeperiod_utc']<pd.to_datetime('2023-02-07')) &
    (ami_df['aep_srvc_qlty_idntfr']=='TOTAL')
]

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='starttimeperiod_utc', y='value', hue='serialnumber', data=tmp_df.reset_index())

In [ ]:
EVs_SNs = mp_df[mp_df['srvc_addr_2_nm'].isin(srvc_addr_2_nms_EVs)]['mfr_devc_ser_nbr'].unique().tolist()
neighbors_SNs = mp_df[mp_df['srvc_addr_2_nm'].isin(srvc_addr_2_nms_neighbors)]['mfr_devc_ser_nbr'].unique().tolist()

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='starttimeperiod_utc', y='value', hue='serialnumber', data=tmp_df[tmp_df['serialnumber'].isin(EVs_SNs)].reset_index())

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='starttimeperiod_utc', y='value', hue='serialnumber', data=tmp_df[tmp_df['serialnumber'].isin(neighbors_SNs)].reset_index())

In [ ]:
tmp_df = ami_df_resamples['12H']['df'].copy()
# tmp_df = tmp_df[
#     (tmp_df['date']>pd.to_datetime('2023-03-01')) &
#     (tmp_df['date']<pd.to_datetime('2023-03-07')) 
# ]

In [ ]:
tmp_df

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='mean_TRS value', hue='serialnumber', data=tmp_df.reset_index())

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='mean_TRS value', hue='serialnumber', data=tmp_df[tmp_df['serialnumber'].isin(EVs_SNs)].reset_index())

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='mean_TRS value', hue='serialnumber', data=tmp_df[tmp_df['serialnumber'].isin(neighbors_SNs)].reset_index())

In [ ]:
list(mp_df.groupby(['mfr_devc_ser_nbr', 'srvc_addr_2_nm']).groups.keys())

In [ ]:
878578506

In [ ]:
tmp_df['serialnumber']=='878578506'

In [ ]:
tmp_df = ami_df_resamples['2H']['df'].copy()
tmp_df = tmp_df[
#     (tmp_df['date']>pd.to_datetime('2023-01-02')) &
#     (tmp_df['date']<pd.to_datetime('2023-01-04')) & 
    (tmp_df['serialnumber']=='878578506')
]

In [ ]:
tmp_df

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='mean_TRS value', hue='serialnumber', data=tmp_df)
ax.grid(True, which='both')

In [ ]:
tmp_df=tmp_df.sort_index()

In [ ]:
tmp_df['mean_TRS value'].tolist()

# SEE WEBPAGES
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data/43512887#43512887
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data/56451135#56451135

In [ ]:
# class real_time_peak_detection():
#     def __init__(self, array, lag, threshold, influence):
#         self.y = list(array)
#         self.length = len(self.y)
#         self.lag = lag
#         self.threshold = threshold
#         self.influence = influence
#         self.signals = [0] * len(self.y)
#         self.filteredY = np.array(self.y).tolist()
#         self.avgFilter = [0] * len(self.y)
#         self.stdFilter = [0] * len(self.y)
#         self.avgFilter[self.lag - 1] = np.mean(self.y[0:self.lag]).tolist()
#         self.stdFilter[self.lag - 1] = np.std(self.y[0:self.lag]).tolist()

#     def thresholding_algo(self, new_value):
#         self.y.append(new_value)
#         i = len(self.y) - 1
#         self.length = len(self.y)
#         if i < self.lag:
#             return 0
#         elif i == self.lag:
#             self.signals = [0] * len(self.y)
#             self.filteredY = np.array(self.y).tolist()
#             self.avgFilter = [0] * len(self.y)
#             self.stdFilter = [0] * len(self.y)
#             self.avgFilter[self.lag] = np.mean(self.y[0:self.lag]).tolist()
#             self.stdFilter[self.lag] = np.std(self.y[0:self.lag]).tolist()
#             return 0

#         self.signals += [0]
#         self.filteredY += [0]
#         self.avgFilter += [0]
#         self.stdFilter += [0]

#         if abs(self.y[i] - self.avgFilter[i - 1]) > (self.threshold * self.stdFilter[i - 1]):

#             if self.y[i] > self.avgFilter[i - 1]:
#                 self.signals[i] = 1
#             else:
#                 self.signals[i] = -1

#             self.filteredY[i] = self.influence * self.y[i] + \
#                 (1 - self.influence) * self.filteredY[i - 1]
#             self.avgFilter[i] = np.mean(self.filteredY[(i - self.lag):i])
#             self.stdFilter[i] = np.std(self.filteredY[(i - self.lag):i])
#         else:
#             self.signals[i] = 0
#             self.filteredY[i] = self.y[i]
#             self.avgFilter[i] = np.mean(self.filteredY[(i - self.lag):i])
#             self.stdFilter[i] = np.std(self.filteredY[(i - self.lag):i])

#         return self.signals[i]


def thresholding_algo(y, lag, threshold, influence):
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    for i in range(lag, len(y)):
        if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter [i-1]:
            if y[i] > avgFilter[i-1]:
                signals[i] = 1
            else:
                signals[i] = -1

            filteredY[i] = influence * y[i] + (1 - influence) * filteredY[i-1]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        else:
            signals[i] = 0
            filteredY[i] = y[i]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [ ]:
rtpd = thresholding_algo(
    y=tmp_df['mean_TRS value'].tolist(), 
    lag=24,
    threshold=10, 
    influence=0.5
)
tmp_df['signals'] = rtpd['signals']

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='mean_TRS value', hue='serialnumber', data=tmp_df)
sns.scatterplot(ax=ax, x='date', y='signals', hue='serialnumber', data=tmp_df, palette='hls')
ax.grid(True, which='both')

In [ ]:
tmp_df = ami_df_resamples['2H']['df'].copy()
tmp_df = tmp_df[
    (tmp_df['serialnumber']=='878578505')
]

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='mean_TRS value', hue='serialnumber', data=tmp_df)
ax.grid(True, which='both')

In [ ]:
rtpd = thresholding_algo(
    y=tmp_df['mean_TRS value'].tolist(), 
    lag=24,
    threshold=7.5, 
    influence=0.5
)
tmp_df['signals'] = rtpd['signals']

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='mean_TRS value', hue='serialnumber', data=tmp_df)
sns.scatterplot(ax=ax, x='date', y='signals', hue='serialnumber', data=tmp_df, palette='hls')
ax.grid(True, which='both')

In [ ]:
tmp_df = ami_df_resamples['2H']['df'].copy()
tmp_df = tmp_df[
    (tmp_df['serialnumber']=='755060041')
]

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='mean_TRS value', hue='serialnumber', data=tmp_df)
ax.grid(True, which='both')

In [ ]:
rtpd = thresholding_algo(
    y=tmp_df['mean_TRS value'].tolist(), 
    lag=24,
    threshold=5, 
    influence=0.5
)
tmp_df['signals'] = rtpd['signals']

In [ ]:
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='mean_TRS value', hue='serialnumber', data=tmp_df)
sns.scatterplot(ax=ax, x='date', y='signals', hue='serialnumber', data=tmp_df, palette='hls')
ax.grid(True, which='both')
Plot_General.set_general_plotting_args(ax=ax, title_args=dict(label='879866273'))

In [ ]:
fig_num=0
for SN_i in EVs_SNs:
    tmp_df = ami_df_resamples['2H']['df'].copy()
    tmp_df = tmp_df[
        (tmp_df['serialnumber']==SN_i)
    ]
    #-----
    rtpd = thresholding_algo(
        y=tmp_df['mean_TRS value'].tolist(), 
        lag=24,
        threshold=7.5, 
        influence=0.5
    )
    tmp_df['signals'] = rtpd['signals']
    #-----
    fig, ax = Plot_General.default_subplots(fig_num=fig_num)
    sns.lineplot(ax=ax, x='date', y='mean_TRS value', hue='serialnumber', data=tmp_df)
    sns.scatterplot(ax=ax, x='date', y='signals', hue='serialnumber', data=tmp_df, palette='hls')
    ax.grid(True, which='both')
    Plot_General.set_general_plotting_args(ax=ax, title_args=dict(label=SN_i))
    fig_num += 1